In [1]:
import csv
import nltk
import re
import numpy as np
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score


DATA_PATH="../data/"
TRAIN_DATA = DATA_PATH + "train_en.tsv"
TEST_DATA = DATA_PATH + "dev_en.tsv"

In [2]:
def readData(path):
    data = []
    with open(path,'r') as file:
        data = [x for x in csv.reader(file, delimiter='\t')]
    return data

def getTweets(raw):
    data = [x[1] for x in raw if x[2] == '1']
    return np.array(data)

def getTarget(raw):
    classes = [x[3] for x in raw if x[2] == '1']
    return np.array(classes)

def getAggression(raw):
    classes = [x[4] for x in raw if x[2] == '1']
    return classes

def removePattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for x in r:
        tweet = re.sub(x, '', tweet)
    return tweet

def preprocess(data):
    cleanData = []
    for tweet in data:
        tweet = removePattern(tweet, "@[\w]*")
        tweet = tweet.replace("#", "") # Removing '#' from hashtags
        tweet = re.sub(r'[^a-zA-Z]', " ", tweet) # Removing punctuation and special characters
        tweet = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',"<URL>", tweet)
        tweet = re.sub(" +", " ", tweet)
        tweet = tweet.lower()
        cleanData.append(tweet)
    return cleanData

def tokenize(text):
    return TweetTokenizer.tokenize(text)

def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='weighted')
    acc = accuracy_score(target, predicted)
    rec = recall_score(target, predicted, average = 'macro')
    print("F1 score:   ", f1)
    print("Avg Recall: ", rec)    
    print("Accuracy:   ", acc)

In [3]:
en_stopwords = set(stopwords.words("english")) 

raw_train = readData(TRAIN_DATA) 
train_tweets = getTweets(raw_train)
train_tweets = preprocess(train_tweets)
Y_train_aggression = getAggression(raw_train)
Y_train_target = getTarget(raw_train)

raw_test = readData(TEST_DATA)
test_tweets = getTweets(raw_test)
test_tweets = preprocess(test_tweets)
Y_test_aggression = getAggression(raw_test)
Y_test_target = getTarget(raw_test)



## Data Analysis

In [4]:
# train_tweets_count = len(train_tweets)
# test_tweets_count = len(test_tweets)
# # print("total number of train tweets: %s", train_tweets_count)

# train_tweets_with_aggression = np.sum(Y_train_aggression)
# test_tweets_with_aggression = np.sum(Y_test_aggression)


# train_tweets_without_aggression = train_tweets_count - train_tweets_with_aggression
# test_tweets_without_aggression = test_tweets_count - test_tweets_with_aggression



## Embeddings Word  Level and Char  Level

In [5]:
def embeddings(typ):
    if typ == "word":
        vectorizer = CountVectorizer(
            analyzer = 'word',
            lowercase = True,
            ngram_range=(1, 1),
            stop_words = en_stopwords)
        vectorizer.fit(train_tweets)
    else:
        vectorizer = CountVectorizer(
            analyzer = 'char',
            tokenizer = tokenize,
            lowercase = True,
            ngram_range=(2, 6),
            stop_words = en_stopwords)
        vectorizer.fit(train_tweets)
    return vectorizer


## Classifier

In [6]:
def model(name):
    if name == "LR":
        classifier = LogisticRegression(C=0.1, solver='sag')
    else:
        classifier = SVC(C = 0.1)
    return classifier


## Task Type

In [7]:
def task_type(name):
    if name == "TC":
        classifier.fit(train_features, Y_train_target)
        y_predict = classifier.predict(test_features)
        evaluate(Y_test_target, y_predict)
    else:
        classifier.fit(train_features, Y_train_aggression)
        y_predict = classifier.predict(test_features)
        evaluate(Y_test_aggression, y_predict)

# Task 1: Target Classification

## Word Level and Logistic Regression

In [8]:
vectorizer = embeddings("word")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("LR")
task_type("TC")

F1 score:    0.8919342124852587
Avg Recall:  0.8940112399016509
Accuracy:    0.892271662763466


/home/vaibhav/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Char Level and Logistic Regression

In [ ]:
vectorizer = embeddings("char")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("LR")
task_type("TC")

F1 score:    0.9063170196296517
Avg Recall:  0.9069854232525465
Accuracy:    0.9063231850117096


## Word Level and SVM

In [ ]:
vectorizer = embeddings("word")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("SVM")
task_type("TC")

/home/vaibhav/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


F1 score:    0.31912076564199965
Avg Recall:  0.5
Accuracy:    0.48711943793911006


/home/vaibhav/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Char Level and SVM

In [ ]:
vectorizer = embeddings("char")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("SVM")
task_type("TC")

F1 score:    0.31912076564199965
Avg Recall:  0.5
Accuracy:    0.48711943793911006


# Task 2: Aggression Detection

## Word Level and Logistic Regression

In [ ]:
vectorizer = embeddings("word")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("LR")
task_type("AD")

/home/vaibhav/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


F1 score:    0.6613086382884759
Avg Recall:  0.660006154928339
Accuracy:    0.6627634660421545


## Char Level and Logistic Regression

In [ ]:
vectorizer = embeddings("char")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("LR")
task_type("AD")

F1 score:    0.6714586627966254
Avg Recall:  0.6702277323485448
Accuracy:    0.6721311475409836


## Word Level and SVM

In [ ]:
vectorizer = embeddings("word")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("SVM")
task_type("AD")

F1 score:    0.3583426409655918
Avg Recall:  0.5
Accuracy:    0.522248243559719


## Char Level and SVM

In [ ]:
vectorizer = embeddings("char")
train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)
classifier = model("SVM")
task_type("AD")